In [1]:
df_data = spark.read.format('org.apache.spark.sql.execution.datasources.csv.CSVFileFormat')\
.option('header', 'true').load('Melbourne.csv')

df_data.show(5)

+---+----------+---------------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+---+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|_c0|    Suburb|        Address|Rooms|Type|    Price|Method|SellerG|     Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|       CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|
+---+----------+---------------+-----+----+---------+------+-------+---------+--------+--------+--------+--------+---+--------+------------+---------+------------------+---------+----------+--------------------+-------------+
|  2|Abbotsford|25 Bloomburg St|    2|   h|1035000.0|     S| Biggin|4/02/2016|     2.5|  3067.0|     2.0|     1.0|0.0|   156.0|        79.0|   1900.0|Yarra City Council| -37.8079|  144.9934|Northern Metropol...|       4019.0|
|  4|Abbotsford|   5 Charles St|    3|   h|1465000.0|    SP| Biggin|4/03/2017|     2.5|  3067.0|

In [2]:
df_data.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: string (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: string (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: string (nullable = true)
 |-- Bathroom: string (nullable = true)
 |-- Car: string (nullable = true)
 |-- Landsize: string (nullable = true)
 |-- BuildingArea: string (nullable = true)
 |-- YearBuilt: string (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: string (nullable = true)



In [3]:
df_data.count()

8577

In [28]:
df_data.describe('Distance','Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea').show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|          Distance|          Bedroom2|          Bathroom|               Car|          Landsize|     BuildingArea|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|              8577|              8577|              8577|              8577|              8577|             8577|
|   mean|11.288818934359051|3.1192724728926198| 1.658621895767751|1.7098052932260697| 528.8772298006296|153.5990767867553|
| stddev| 6.836571004106779|0.9302257752990112|0.7242505221888721|0.9794236482240574|1071.6671390054103|86.37989456829557|
|    min|               0.0|               1.0|               1.0|               0.0|               0.0|            100.0|
|    max|               9.9|               9.0|               9.0|               9.0|             999.0|             99.5|
+-------+-------

In [7]:
df_data.cache()

DataFrame[_c0: string, Suburb: string, Address: string, Rooms: string, Type: string, Price: string, Method: string, SellerG: string, Date: string, Distance: string, Postcode: string, Bedroom2: string, Bathroom: string, Car: string, Landsize: string, BuildingArea: string, YearBuilt: string, CouncilArea: string, Lattitude: string, Longtitude: string, Regionname: string, Propertycount: string]

In [29]:
from pyspark.sql.types import DoubleType
filtered_typed_df=df_data.withColumn("Bedroom2", df_data["Bedroom2"].cast("double"))\
.withColumn("Bathroom", df_data["Bathroom"].cast("double"))\
.withColumn("Car", df_data["Car"].cast("double"))\
.withColumn("Landsize", df_data["Landsize"].cast("double"))\
.withColumn("BuildingArea", df_data["BuildingArea"].cast("double"))\
.withColumn("Price", df_data["Price"].cast("double"))\
.withColumn("Distance", df_data["Distance"].cast("double"))
filtered_typed_df.printSchema()

root
 |-- _c0: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: string (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: string (nullable = true)



In [30]:
filtered_typed_df.describe('Distance','Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea').show()

+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|summary|          Distance|          Bedroom2|          Bathroom|               Car|          Landsize|     BuildingArea|
+-------+------------------+------------------+------------------+------------------+------------------+-----------------+
|  count|              8577|              8577|              8577|              8577|              8577|             8577|
|   mean|11.288818934359051|3.1192724728926198| 1.658621895767751|1.7098052932260697| 528.8772298006296|153.5990767867553|
| stddev| 6.836571004106779|0.9302257752990112|0.7242505221888721|0.9794236482240574|1071.6671390054103|86.37989456829557|
|    min|               0.0|               1.0|               1.0|               0.0|               0.0|             50.0|
|    max|              47.4|              12.0|               9.0|              10.0|           42800.0|           3112.0|
+-------+-------

In [31]:
from pyspark.ml.feature import VectorAssembler
#put together the relevant elements in a vector
assembler1 = VectorAssembler(inputCols=['Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea'], 
                             outputCol='features')
#add the vector as an additional column
filtered_features_df = assembler1.transform(filtered_typed_df)
filtered_features_df.printSchema()
filtered_features_df.show(5) 

root
 |-- _c0: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: string (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: string (nullable = true)
 |-- features: vector (nullable = true)

+---+----------+---------------+-----+----+---------+------+-------+---------+--------

In [32]:
#we need to scale all the features to avoid having instability. It considers mean and std
from pyspark.ml.feature import StandardScaler
#indicate which column you want to scale
scaler1 = StandardScaler(inputCol="features", outputCol="scaled_features")

# Compute summary statistics and generate StandardScalerModel 
scaler_model1 = scaler1.fit(filtered_features_df)

# rescale each feature
scaled_data_df = scaler_model1.transform(filtered_features_df)

scaled_data_df.printSchema()

scaled_data_df.select("features", "scaled_features").show()

root
 |-- _c0: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: string (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)

+--------------------+-----------------

In [33]:
#now we have a proper dataset to use for building the model
#extract a trainig set and test set from the dataset randomly (80% training and 20% test)
training_data_df, testing_data_df = scaled_data_df.randomSplit([.8,.2],seed=1234)
training_data_df.show(5)

+-----+-------------+----------------+-----+----+---------+------+-------------+----------+--------+--------+--------+--------+---+--------+------------+---------+--------------------+---------+----------+--------------------+-------------+--------------------+--------------------+
|  _c0|       Suburb|         Address|Rooms|Type|    Price|Method|      SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|         CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|            features|     scaled_features|
+-----+-------------+----------------+-----+----+---------+------+-------------+----------+--------+--------+--------+--------+---+--------+------------+---------+--------------------+---------+----------+--------------------+-------------+--------------------+--------------------+
| 1001|       Balwyn|3/12 Brenbeal St|    2|   u| 730000.0|     S|hockingstuart|28/08/2016|     9.7|  3103.0|     2.0|     1.0|1.0|   130.0|        80.

In [34]:
testing_data_df.show(5)

+-----+-------------+-----------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+--------------------+---------+----------+--------------------+-------------+--------------------+--------------------+
|  _c0|       Suburb|          Address|Rooms|Type|    Price|Method| SellerG|      Date|Distance|Postcode|Bedroom2|Bathroom|Car|Landsize|BuildingArea|YearBuilt|         CouncilArea|Lattitude|Longtitude|          Regionname|Propertycount|            features|     scaled_features|
+-----+-------------+-----------------+-----+----+---------+------+--------+----------+--------+--------+--------+--------+---+--------+------------+---------+--------------------+---------+----------+--------------------+-------------+--------------------+--------------------+
|  100| Airport West|     180 Parer Rd|    3|   h| 830000.0|     S|   Barry|16/04/2016|    13.5|  3042.0|     3.0|     1.0|2.0|   971.0|       113.0|   1960.0|Moon

In [35]:
#run linear regression on the training set to learn the model
from  pyspark.ml.regression import LinearRegression
#run training specifying the features to use and the label - we want to predict the price from beds, baths, and sq_ft
lr = LinearRegression(featuresCol = 'scaled_features', labelCol = 'Price')
#lr = LinearRegression(featuresCol = 'features', labelCol = 'price')

# Fit the model
lr_model = lr.fit(training_data_df)
print("Coefficients: %s" % str(lr_model.coefficients))
print("Intercept: %s" % str(lr_model.intercept))

Coefficients: [-262165.7608028521,150973.0771698868,86240.48713542748,24728.86197643539,27922.48839538722,256043.44064674916]
Intercept: 324053.2853304953


In [36]:
# Summarize the model over the training set and print out some metrics
training_summary = lr_model.summary
print("RMSE: %f" % training_summary.rootMeanSquaredError)

RMSE: 503872.440720


In [37]:
#evaluate the model on the test set
testing_summary = lr_model.evaluate(testing_data_df)
testing_summary.predictions.printSchema()
testing_summary.predictions.select('Distance', 'Bedroom2', 'Bathroom', 'Car', 'Landsize', 'BuildingArea','Price','prediction').show(10)

root
 |-- _c0: string (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- Rooms: string (nullable = true)
 |-- Type: string (nullable = true)
 |-- Price: double (nullable = true)
 |-- Method: string (nullable = true)
 |-- SellerG: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Distance: double (nullable = true)
 |-- Postcode: string (nullable = true)
 |-- Bedroom2: double (nullable = true)
 |-- Bathroom: double (nullable = true)
 |-- Car: double (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- BuildingArea: double (nullable = true)
 |-- YearBuilt: string (nullable = true)
 |-- CouncilArea: string (nullable = true)
 |-- Lattitude: string (nullable = true)
 |-- Longtitude: string (nullable = true)
 |-- Regionname: string (nullable = true)
 |-- Propertycount: string (nullable = true)
 |-- features: vector (nullable = true)
 |-- scaled_features: vector (nullable = true)
 |-- prediction: double (nullable = fals